In [1]:
library(tidyverse)
library(TCGAbiolinks)
library(HDF5Array)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DelayedArray
Loading required package: stats4
Loading required package: matrixStats

Attaching package: ‘matrixStats’

The following object is masked from ‘package:dplyr’:

    count

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    co

# Constants

In [6]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-OV", "TCGA-UCS", "TCGA-UCEC")
project_paths <- unlist(map(projects, function(prj) paste0(dirs$data_dir, "/", prj)))

# Functions

In [43]:
protein_expr_query <- function(p) {
    return(
        GDCquery(
            project = p,
            data.category = "Protein expression",
            legacy = TRUE
        )
    )
}


get_participant_id <- function(barcode) {
    return(
        unlist(str_split(barcode, "-"))[3]
    )
}

In [18]:
proj_idx <- 1
q <- protein_expr_query(projects[proj_idx])
GDCdownload(q, method = "api", directory = paste0(dirs$data_dir, "/tcga_biolinks_downloads"), files.per.chunk = 10)
data <- GDCprepare(q, directory = paste0(dirs$data_dir, "/tcga_biolinks_downloads"))

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg19
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-CESC
--------------------
oo Filtering results
--------------------
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------
GDCdownload will download 173 files. A total of 1.004929 MB


Downloading: 21 kB      

Downloading: 21 kB      

Downloading: 22 kB      

Downloading: 22 kB      

...

Downloading: 21 kB      

Downloading: 21 kB      

Downloading: 21 kB      

Downloading: 21 kB      

  |======================================================================| 100%


In [20]:
coldata_df <- read_tsv(paste0(dirs$data_dir, "/unified_cervical_data/coldata.tsv"))

Parsed with column specification:
cols(
  sample_name = col_character(),
  condition = col_character(),
  data_source = col_character()
)


In [52]:
RNA_sample_names <- (coldata_df %>%
    dplyr::filter(data_source == "TCGA"))$sample_name

In [33]:
colnames(data[, -1])[1:10]

[1] "TCGA-EA-A439-01A-21-A40I-20" "TCGA-VS-A9UB-01A-11-A40H-20"
 [3] "TCGA-DS-A1OA-01A-21-A40H-20" "TCGA-LP-A5U3-01A-21-A40I-20"
 [5] "TCGA-MA-AA42-01A-21-A40H-20" "TCGA-EA-A43B-01A-71-A40I-20"
 [7] "TCGA-EA-A411-01A-21-A40I-20" "TCGA-LP-A7HU-01A-21-A40H-20"
 [9] "TCGA-FU-A2QG-01A-21-A40H-20" "TCGA-C5-A2LX-01A-21-A40H-20"

In [53]:
RNA_participants <- map(.f = get_participant_id, RNA_sample_names)
protein_participants <- map(.f = get_participant_id, colnames(data[, -1]))

In [58]:
length(intersect(RNA_participants, protein_participants))

[1] 145